In [1]:
import requests
import json
import time
import zipfile
import os
import shutil

# === CONFIG ===
email = "your_email@example.com"  # <-- Replace with your real email
priority_code = "K3LS9F"
area_of_interest = "-118.2211 34.2608 -117.9437 34.1417"
resolution = "31"  # 30m recommended for FlamMap
output_zip = "eaton_landscape.zip"
extract_dir = "eaton_landscape_extracted"

# === REQUEST ALL REQUIRED LAYERS FOR LCP ===
layer_list = "ELEV2020;SLPD2020;ASP2020;230FBFM40;230CC;230CH;230CBH;230CBD"

# === SUBMIT JOB ===
submit_url = "https://lfps.usgs.gov/api/job/submit"
headers = {"accept": "application/json", "Content-Type": "application/json"}

payload = {
    "Email": email,
    "Layer_List": layer_list,
    "Area_of_Interest": area_of_interest,
    "Output_Projection": "5070",  
    "Resample_Resolution": resolution,
    "Edit_Rule": {},
    "Edit_Mask": "",
    "Priority_Code": priority_code
}


print(" Submitting job to LANDFIRE...")
response = requests.post(submit_url, headers=headers, data=json.dumps(payload))
print("Status Code:", response.status_code)

if response.status_code != 200:
    print(" Error submitting job.")
    print(response.text)
    exit()

job_info = response.json()
job_id = job_info.get("jobId")

if not job_id:
    print(" No jobId returned.")
    exit()

print(" Job ID:", job_id)
status_url = f"https://lfps.usgs.gov/api/job/status?JobId={job_id}"
print(" Polling for job status...")

# === POLL UNTIL READY ===
while True:
    status_response = requests.get(status_url)
    try:
        status_data = status_response.json()
    except Exception as e:
        print(" Error parsing job status:", e)
        print("Raw:", status_response.text)
        break

    job_status = status_data.get("status")
    print(" Job Status:", job_status)

    if job_status == "Succeeded":
        download_url = status_data.get("outputFile")
        if not download_url:
            print(" Job succeeded but no download URL found.")
            break

        print(" Downloading:", download_url)
        file_resp = requests.get(download_url)
        with open(output_zip, "wb") as f:
            f.write(file_resp.content)
        print(f" Downloaded ZIP to {output_zip}")
        break

    elif job_status == "Failed":
        print(" Job failed.")
        break

    time.sleep(15)

# === UNZIP THE OUTPUT ===
print(f" Extracting {output_zip}...")
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f" Extracted to {extract_dir}")

# === SEARCH FOR .LCP FILE ===
print(" Looking for .lcp file...")
lcp_file = None
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        if file.lower().endswith(".lcp"):
            lcp_file = os.path.join(root, file)
            break

if lcp_file:
    final_name = "eaton.lcp"
    shutil.copy(lcp_file, final_name)
    print(f" LCP file copied to {final_name}")
else:
    print(" No .lcp file found. Check the extracted contents.")


 Submitting job to LANDFIRE...
Status Code: 200
 Job ID: a384ef10-ffb5-438a-b3cc-5545b44767d1
 Polling for job status...
 Job Status: Pending
 Job Status: Executing
 Job Status: Succeeded
 Downloading: https://lfps.usgs.gov/arcgis/rest/directories/arcgisjobs/landfireproductservice_gpserver/jf0ee3ec3fec7498aa54a1b71fd592fec/scratch/jf0ee3ec3fec7498aa54a1b71fd592fec.zip
 Downloaded ZIP to eaton_landscape.zip
 Extracting eaton_landscape.zip...
 Extracted to eaton_landscape_extracted
 Looking for .lcp file...
 No .lcp file found. Check the extracted contents.
